In [2]:
# planner_chain에서 미리 어떤 명령어들이 있는지 숙지한 후에 plan을 세우도록 유도
# show_task를 진행할 때, execute가 결과를 낼 때, task_id를 같이 보
# 여줘서 이 이후에 이를 보고 작업을 할 수 있도록 프롬프트 수정 필요

In [12]:
from typing import Literal, Annotated, List
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, RemoveMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.graph import END

from dotenv import load_dotenv
import os

from langchain.agents import Tool
from langchain.tools import StructuredTool
from notion.planner import Planner, ConfirmablePlanner
from notion.task import Task
from pydantic import BaseModel, Field

from datetime import datetime, timedelta
import json

load_dotenv()

# 메모리 저장소 설정
memory = MemorySaver()

planner = Planner()
# planner = ConfirmablePlanner()

# 메시지 상태와 요약 정보를 포함하는 상태 클래스
class State(MessagesState):
    messages: Annotated[list, add_messages]
    summary: str


# 대화 및 요약을 위한 모델 초기화
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 기본 타입 베이스
class NameSchema(BaseModel):
    name: Annotated[str, "A task name"]

class GroupSchema(BaseModel):
    group_name: Annotated[str, "A group name"]

class DatetimeSchema(BaseModel):
    date: Annotated[str, "Datetime string, format: YYYY-MM-DD HH:MM"]

class DateSchema(BaseModel):
    start_datetime: Annotated[DatetimeSchema, "Start datetime of the task"]
    end_datetime: Annotated[DatetimeSchema, "End datetime of the task"]

class TaskSchema(BaseModel):
    name: Annotated[NameSchema, "A new task name to create"]
    date: Annotated[DateSchema, "A new task date to create. if there are single datetime, make end_datetime as same as start_datetime"]
    group: Annotated[GroupSchema, "A new task group to create"]

# planner 도구
class ShowTasksSchema(BaseModel):
    pass

def show_tasks():
    return planner.show_tasks()

class ShowTasksByGroupSchema(BaseModel):
    group: Annotated[GroupSchema, "The group to filter tasks"]

def show_tasks_by_group(group: str):
    return planner.show_tasks_by_group(group)

class ShowTasksAfterNowSchema(BaseModel):
    pass

def show_tasks_after_now():
    return planner.show_tasks_after_now()

class CreateTaskSchema(BaseModel):
    name: str = Field(..., description="Task name")
    start_date: str = Field(..., description="""The start datetime of the task, format: 'YYYY-MM-DD HH:MM'""")
    end_date: str = Field(None, description="The end datetime of the task, format: 'YYYY-MM-DD HH:MM'. If not provided, the task is considered to have no end date.")
    group: str = Field(..., description="Task group, e.g., '일상', '이벤트'")

def create_task(name, start_date, end_date, group):
    if end_date:
        date = {"start": start_date, "end": end_date}
    else:
        date = {"start": start_date, "end": None}

    task = Task(task_id=None, name=name, date=date, group=group)
    return planner.add_task(task)

# def create_task(name, date, group):
#     try:
#         date_count = date.count(":")
#         if date_count == 1:
#             date = {"start": date, "end": None}
#         elif date_count == 2:
#             part = date.split(" ")
#             start = f"{part[0]} {part[1]}"
#             end = f"{part[2]} {part[3]}"
#             date = {"start": start, "end": end}
#         else:
#             raise ValueError("Invalid date format. Use 'YYYY-MM-DD HH:MM' or 'YYYY-MM-DD HH:MM YYYY-MM-DD HH:MM'.")
#     except ValueError as e:
#         raise ValueError(f"Error parsing date: {e}")

#     task = Task(task_id=None, name=name, date=date, group=group)
#     return planner.add_task(task)


class CreateTasksSchema(BaseModel):
    tasks: str = Field(..., description="""A JSON string representing a list of tasks. Each task should be a list with the format: [name, date, group].
                        name: Task name
                        date: The date of the task, either format:
                        - Single datetime: 'YYYY-MM-DD HH:MM'
                        - Start and end datetime: 'YYYY-MM-DD HH:MM YYYY-MM-DD HH:MM'
                        group: Task group, e.g., '일상', '이벤트'
                       
                    Example: '[["Task 1", "2023-10-01 12:00", "일상"], ["Task 2", "2023-10-02 14:00", "이벤트"]]'""")

def create_tasks(tasks):
    try:
        str_task_list = json.loads(tasks)

        if not isinstance(str_task_list, list):
            raise ValueError("Tasks should be a list of task.")
        
        tasks = []
        for str_task in str_task_list:
            if not isinstance(str_task, list):
                raise ValueError("Each task should be a list.")
            
            name = str_task[0]
            date = str_task[1]
            group = str_task[2]
            task = Task(task_id=None, name=name, date=date, group=group)
            tasks.append(task)
        
        return planner.add_tasks(tasks)
    
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid format: {e}")
    

class DeleteTaskSchema(BaseModel):
    task_id: str = Field(..., description="The ID of the task to delete")

def delete_task(task_id):
    return planner.delete_task(task_id)

class DeleteTasksSchema(BaseModel):
    task_ids: Annotated[List[str], """A list of task IDs to delete. Example: '["task_id_1", "task_id_2"]'"""]

def delete_tasks(task_ids):
    return planner.delete_tasks(task_ids)

class EditTaskSchema(BaseModel):
    task_id: str = Field(..., description="The ID of the task to edit")
    name: str = Field("No name", description="New task name")
    date: str = Field("2000-01-01 00:00", description="New date of the task, either format:\n- Single datetime: 'YYYY-MM-DD HH:MM'\n- Start and end datetime: 'YYYY-MM-DD HH:MM YYYY-MM-DD HH:MM'")
    group: str = Field("no group", description="New task group, e.g., '일상', '이벤트'")

def edit_task(task_id, name, date, group):
    new_task = Task(task_id=task_id, name=name, date=date, group=group)
    return planner.edit_task(task_id, new_task)

class EditTasksSchema(BaseModel):
    tasks: str = Field(..., description="""A JSON string representing a list of tasks to edit. Each task should be a list with the format: [task_id, name, date, group].
                        task_id: The ID of the task to edit
                        name: New task name
                        date: New date of the task, either format:
                        - Single datetime: 'YYYY-MM-DD HH:MM'
                        - Start and end datetime: 'YYYY-MM-DD HH:MM YYYY-MM-DD HH:MM'
                        group: New task group, e.g., '일상', '이벤트'
                       
                    Example: '[["task_id_1", "New Task 1", "2023-10-01 12:00", "일상"], ["task_id_2", "New Task 2", "2023-10-02 14:00", "이벤트"]]'""")
    
def edit_tasks(tasks):
    try:
        str_task_list = json.loads(tasks)

        if not isinstance(str_task_list, list):
            raise ValueError("Tasks should be a list of task.")
        
        tasks = {}
        for str_task in str_task_list:
            if not isinstance(str_task, list):
                raise ValueError("Each task should be a list.")
            
            task_id = str_task[0]
            name = str_task[1]
            date = str_task[2]
            group = str_task[3]
            task = Task(task_id=task_id, name=name, date=date, group=group)
            tasks[task_id] = task
        
        return planner.edit_tasks(tasks)
    
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid format: {e}")


# 기타 도구
class GetNowDatetimeSchema(BaseModel):
    pass

def get_now_datetime():
    return datetime.now().astimezone().strftime('%Y-%m-%d %H:%M %A')

class CalculateDatetimeWithNumberOfDaysSchema(BaseModel):
    date: str = Field(..., description="The date to calculate from, in the format 'YYYY-MM-DD HH:MM'.")
    number_of_days: int = Field(..., description="The number of days to add or subtract. Use negative values to subtract days.")

def calculate_datetime_with_number_of_days(date: str, number_of_days: int):
    try:
        # Parse the input date
        dt = datetime.strptime(date, '%Y-%m-%d %H:%M')
        # Add or subtract the number of days
        new_dt = dt + timedelta(days=number_of_days)
        # Return the new date in the desired format
        return new_dt.strftime('%Y-%m-%d %H:%M %A')
    except ValueError as e:
        raise ValueError(f"Invalid date format: {e}")

class GetAllDatesBetweenSchema(BaseModel):
    start_date: str = Field(..., description="The start date in the format 'YYYY-MM-DD HH:MM'.")
    end_date: str = Field(..., description="The end date in the format 'YYYY-MM-DD HH:MM'.")

def get_all_dates_between(start_date: str, end_date: str) -> List[str]:
    try:
        start_dt = datetime.strptime(start_date, '%Y-%m-%d %H:%M')
        end_dt = datetime.strptime(end_date, '%Y-%m-%d %H:%M')
        
        if start_dt > end_dt:
            raise ValueError("Start date must be before or equal to end date.")
        
        date_list = []
        current_dt = start_dt
        while current_dt <= end_dt:
            date_list.append(current_dt.strftime('%Y-%m-%d %H:%M %A'))
            current_dt += timedelta(days=1)
        
        return date_list
    except ValueError as e:
        raise ValueError(f"Invalid date format: {e}")

tools = [
    StructuredTool.from_function(
        func=show_tasks,
        name="ShowTasks",
        description="Show all the tasks.",
        args_schema=ShowTasksSchema
    ),
    StructuredTool.from_function(
        func=show_tasks_by_group,
        name="ShowTasksByGroup",
        description="Show tasks filtered by group.",
        args_schema=ShowTasksByGroupSchema
    ),
    StructuredTool.from_function(
        func=show_tasks_after_now,
        name="ShowTasksAfterNow",
        description="Show tasks that are scheduled after the current time.",
        args_schema=ShowTasksAfterNowSchema
    ),
    StructuredTool.from_function(
        func=create_task,
        name="CreateTask",
        description="Create/Add a new task in Notion.",
        args_schema=CreateTaskSchema
    ),
    # StructuredTool.from_function(
    #     func=create_tasks,
    #     name="CreateTasks",
    #     description="Create/Add multiple tasks in Notion.",
    #     args_schema=CreateTasksSchema
    # ),
    StructuredTool.from_function(
        func=delete_task,
        name="DeleteTask",
        description="Delete a task from Notion. Input: Notion page ID.",
        args_schema=DeleteTaskSchema
    ),
    StructuredTool.from_function(
        func=delete_tasks,
        name="DeleteTasks",
        description="Delete multiple tasks from Notion. Input: JSON string of task IDs.",
        args_schema=DeleteTasksSchema
    ),
    StructuredTool.from_function(
        func=edit_task,
        name="EditTask",
        description="Update a task's title in Notion. Input: page ID and new title.",
        args_schema=EditTaskSchema
    ),
    StructuredTool.from_function(
        func=edit_tasks,
        name="EditTasks",
        description="Update multiple tasks in Notion. Input: JSON string of task details.",
        args_schema=EditTasksSchema
    ),
    StructuredTool.from_function(
        func=get_now_datetime,
        name="GetNowDatetime",
        description="Get the current date and time in the format 'YYYY-MM-DD HH:MM EEEE'.",
        args_schema=GetNowDatetimeSchema
    ),
    StructuredTool.from_function(
        func=calculate_datetime_with_number_of_days,
        name="CalculateDatetimeWithNumberOfDays",
        description="Calculate a new date by adding or subtracting a number of days from a given date.",
        args_schema=CalculateDatetimeWithNumberOfDaysSchema
    ),
    StructuredTool.from_function(
        func=get_all_dates_between,
        name="GetAllDatesBetween",
        description="Get all dates between two given dates, inclusive.",
        args_schema=GetAllDatesBetweenSchema
    ),
]

def print_stream(stream):
    for chunk in stream:
        message = chunk["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


tool_agent_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Your task is calling a tool as user requested. "
            "Answer in Korean.",
        ),
        ("human", "{messages}"),
    ]
)


from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(
    model=model,
    tools=tools,
    prompt=tool_agent_prompt
    # checkpointer=memory
)

In [14]:
preinform_text = "오늘 날짜: " + get_now_datetime() + "\n"

date_selection = "오늘"

input_texts = [
    f"{date_selection} YYYY-MM-DD 형식으로 날짜 출력",
    "위 날짜를 YYYY-MM-DD 12:00 ~ 18:00 형식으로 출력",
    "위 결과를 바탕으로 일정 생성해줘, 이름은 '알바', 그룹은 '일과'",
]

history = []
history.append(preinform_text)
for input_text in input_texts:
    print("=== User Input ===")
    history.append(input_text)
    print(input_text)
    
    print("=== Agent Response ===")
    response = agent_executor.invoke({"messages": [HumanMessage(content="\n".join(history))]})
    ai_message = response["messages"][-1].content
    history.append(ai_message)
    print(ai_message)

=== User Input ===
오늘 YYYY-MM-DD 형식으로 날짜 출력
=== Agent Response ===
오늘 날짜는 2025-10-28입니다.
=== User Input ===
위 날짜를 YYYY-MM-DD 12:00 ~ 18:00 형식으로 출력
=== Agent Response ===
오늘 날짜는 2025-10-28입니다. 

주어진 시간 범위는 다음과 같습니다:
- 2025-10-28 12:00 Tuesday
- 2025-10-28 18:00 Tuesday

이 날짜와 시간은 YYYY-MM-DD 형식으로 출력되었습니다.
=== User Input ===
위 결과를 바탕으로 일정 생성해줘, 이름은 '알바', 그룹은 '일과'
=== Agent Response ===
일정 '알바'가 성공적으로 추가되었습니다! 일정은 2025년 10월 28일 12:00부터 18:00까지입니다.


In [2]:
def get_tool_descriptions(tools: list) -> str:
    descs = []
    for tool in tools:
        schema = tool.args_schema
        schema_fields = schema.model_fields

        schema_description_lines = []
        for field_name, field_info in schema_fields.items():
            annotation = field_info.annotation
            field_type = getattr(annotation, '__name__', str(annotation))
            description = field_info.description or "No description"
            schema_description_lines.append(f"- `{field_name}` ({field_type}): {description}")

        schema_description = "\n".join(schema_description_lines)

        descs.append(
            f"Tool: {tool.name}\nDescription: {tool.description}\nArguments:\n{schema_description}"
        )
    return "\n\n".join(descs)

available_tools_description = get_tool_descriptions(tools)

In [3]:
"""
config = {"configurable": {"thread_id": "1"}}
inputs = {"messages": [
    ("user", "how much tasks did you saw?")
]}
response = agent_executor.invoke(
    inputs
)
print(response["messages"][-1].content)

config = {"configurable": {"thread_id": "1"}}
inputs = {"messages": [
    ("user", "how much tasks did you saw?")
]}
print_stream(agent_executor.stream(inputs, config=config, stream_mode="values"))
"""

'\nconfig = {"configurable": {"thread_id": "1"}}\ninputs = {"messages": [\n    ("user", "how much tasks did you saw?")\n]}\nresponse = agent_executor.invoke(\n    inputs\n)\nprint(response["messages"][-1].content)\n\nconfig = {"configurable": {"thread_id": "1"}}\ninputs = {"messages": [\n    ("user", "how much tasks did you saw?")\n]}\nprint_stream(agent_executor.stream(inputs, config=config, stream_mode="values"))\n'

In [3]:
import operator
from typing import Annotated, List, Tuple, Union
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


model_name = "gpt-4o"
temperature = 0.2


# 상태 정의
class PlanExecute(TypedDict):
    input: Annotated[str, "User's input"]
    plan: Annotated[List[str], "Current plan"]
    past_steps: Annotated[List[List[str]], "Past Steps, each step is a list of [task, response]"]
    response: Annotated[str, "Final response"]


# Plan 모델 정의
class Plan(BaseModel):
    """Sorted steps to execute the plan"""

    steps: Annotated[List[str], "Different steps to follow, should be in sorted order"]

# 계획 수립을 위한 프롬프트 템플릿 생성
planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Answer in Korean.

Following the tools that are available, Make plan.:
{tools_description}""",
        ),
        ("placeholder", "{messages}"),
    ]
)

planner_chain = planner_prompt | ChatOpenAI(
    model=model_name, temperature=temperature
).with_structured_output(Plan)


class Response(BaseModel):
    """Response to user."""

    # 사용자 응답
    response: str


class Act(BaseModel):
    """Action to perform."""

    # 수행할 작업: "Response", "Plan". 사용자에게 응답할 경우 Response 사용, 추가 도구 사용이 필요할 경우 Plan 사용
    action: Union[Response, Plan] = Field(
        description="Action to perform. If you want to respond to user, use Response. "
        "If you need to further use tools to get the answer, use Plan."
    )


# 계획을 재수립하기 위한 프롬프트 정의
replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Following the tools that are available:
{tools_description}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan.

Answer in Korean."""
)


# Replanner 생성
replanner_chain = replanner_prompt | ChatOpenAI(
    model=model_name, temperature=temperature
).with_structured_output(Act)



from langchain_core.output_parsers import StrOutputParser


# 사용자 입력을 기반으로 계획을 생성하고 반환
def plan_step(state: PlanExecute):
    planner_input = {"tools_description": available_tools_description, "messages": [("user", state["input"])]}
    plan = planner_chain.invoke(planner_input)

    # 생성된 계획의 단계 리스트 반환
    return {"plan": plan.steps}


# 에이전트 실행기를 사용하여 주어진 작업을 수행하고 결과를 반환
def execute_step(state: PlanExecute):
    plan = state["plan"]
    if "past_steps" not in state:
        # 이전 단계가 없으면 빈 리스트로 초기화
        state["past_steps"] = []
        
    past_steps = state["past_steps"]

    # 계획을 문자열로 변환하여 각 단계에 번호를 매김
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]

    # 현재 실행할 작업을 포맷팅하여 에이전트에 전달
    task_formatted = f"""For the following plan: {plan_str}\n\nYou are tasked with executing [{task}]."""
    # 에이전트 실행기를 통해 작업 수행 및 결과 수신

    agent_response = agent_executor.invoke({"messages": [("user", task_formatted)]})
    # 이전 단계와 그 결과를 포함하는 딕셔너리 반환

    past_steps.append([task, agent_response["messages"][-1].content])

    print("past_steps")
    print(past_steps)
    return {
        "past_steps": past_steps,
    }


# 이전 단계의 결과를 바탕으로 계획을 업데이트하거나 최종 응답을 반환
def replan_step(state: PlanExecute):
    state["tools_description"] = available_tools_description
    output = replanner_chain.invoke(state)
    # 응답이 사용자에게 반환될 경우
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    # 추가 단계가 필요할 경우 계획의 단계 리스트 반환
    else:
        next_plan = output.action.steps
        if len(next_plan) == 0:
            return {"response": "No more steps needed."}
        else:
            return {"plan": next_plan, "past_steps": state["past_steps"]}


# 에이전트의 실행 종료 여부를 결정하는 함수
def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return "final_report"
    else:
        return "execute"


final_report_prompt = ChatPromptTemplate.from_template(
    """You are given the objective and the previously done steps. Your task is to generate a final report in markdown format.
Final report should be written in professional tone.

Your objective was this:

{input}

Your previously done steps(question and answer pairs):

{past_steps}

Generate a final report in markdown format. Write your response in Korean."""
)

final_report = (
    final_report_prompt
    | ChatOpenAI(model=model_name, temperature=temperature)
    | StrOutputParser()
)


def generate_final_report(state: PlanExecute):
    past_steps = "\n\n".join(
        [
            f"Question: {past_step[0]}\n\nAnswer: {past_step[1]}\n\n####"
            for past_step in state["past_steps"]
        ]
    )
    response = final_report.invoke({"input": state["input"], "past_steps": past_steps})
    return {"response": response}


# 업데이트 정보 출력 함수
def print_update(update):
    # 업데이트 딕셔너리 순회
    print("-" * 20)
    for k, v in update.items():
        # 메시지 목록 출력

        if "input" in v:
            print(f"Input: {v['input']}")
        
        if "plan" in v:
            for m in v["plan"]:
                print(f"Plan step: {m}")

        if "past_steps" in v:
            for m in v["past_steps"]:
                print(f"Past step: {m}")
        
        if "response" in v:
            print(f"Response: {v['response']}")


from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

    
# 그래프 생성
workflow = StateGraph(PlanExecute)

# 노드 정의
workflow.add_node("planner", plan_step)
workflow.add_node("execute", execute_step)
workflow.add_node("replan", replan_step)
workflow.add_node("final_report", generate_final_report)

# 엣지 정의
workflow.add_edge(START, "planner")
workflow.add_edge("planner", "execute")
workflow.add_edge("execute", "replan")
workflow.add_edge("final_report", END)

# 조건부 엣지: replan 후 종료 여부를 결정하는 함수 사용
workflow.add_conditional_edges(
    "replan",
    should_end,
    {"execute": "execute", "final_report": "final_report"},
)

# 그래프 컴파일
# app = workflow.compile(checkpointer=MemorySaver())
app = workflow.compile()

# print(app.get_graph().draw_ascii())

In [4]:
from langchain_core.runnables import RunnableConfig

# config = RunnableConfig(recursion_limit=50, configurable={"thread_id": 1})

# 메시지 핸들링을 위한 HumanMessage 클래스 임포트
from langchain_core.messages import HumanMessage

# 스레드 ID가 포함된 설정 객체 초기화
config = {"configurable": {"thread_id": "1"}, "recursion_limit": 50}

# 첫 번째 사용자 메시지 생성 및 출력
# input_message = HumanMessage(content="Modular RAG 가 기존의 Naive RAG 와 어떤 차이가 있는지와 production level 에서 사용하는 이점을 설명해줘")
# input_message = HumanMessage(content="오늘은 2025-06-22이다. 내일부터 5일동안 일할거니까, 12:00~18:00 까지 '알바'라는 이름이고 '일상' 그룹에 총 5개의 일정 만들어.")
input_message = HumanMessage(content="'일상' 일정의 기간 보여줘")
input_message.pretty_print()

inputs = {"input": input_message.content}

# 스트림 모드에서 첫 번째 메시지 처리 및 업데이트 출력
for event in app.stream(inputs, config, stream_mode="updates"):
    # print_update(event)
    pass

================================ Human Message =================================

'일상' 일정의 기간 보여줘
past_steps
[["1. 'ShowTasksByGroup' 도구를 사용하여 '일상' 그룹의 모든 일정을 확인한다.", 'Sorry, need more steps to process this request.']]
past_steps
[["1. 'ShowTasksByGroup' 도구를 사용하여 '일상' 그룹의 모든 일정을 확인한다.", 'Sorry, need more steps to process this request.'], ["1. 'ShowTasksByGroup' 도구를 사용하여 '일상' 그룹의 모든 일정을 확인한다.", 'Sorry, need more steps to process this request.']]
past_steps
[["1. 'ShowTasksByGroup' 도구를 사용하여 '일상' 그룹의 모든 일정을 확인한다.", 'Sorry, need more steps to process this request.'], ["1. 'ShowTasksByGroup' 도구를 사용하여 '일상' 그룹의 모든 일정을 확인한다.", 'Sorry, need more steps to process this request.'], ["1. 'ShowTasksByGroup' 도구를 사용하여 '일상' 그룹의 모든 일정을 확인한다.", 'Sorry, need more steps to process this request.']]
past_steps
[["1. 'ShowTasksByGroup' 도구를 사용하여 '일상' 그룹의 모든 일정을 확인한다.", 'Sorry, need more steps to process this request.'], ["1. 'ShowTasksByGroup' 도구를 사용하여 '일상' 그룹의 모든 일정을 확인한다.", 'Sorry, need more steps to process

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-TC1JXZcfi5MBiQBrCWgseKda on tokens per min (TPM): Limit 200000, Used 193800, Requested 7092. Please try again in 267ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [55]:
# 메시지 핸들링을 위한 HumanMessage 클래스 임포트
from langchain_core.messages import HumanMessage

# 스레드 ID가 포함된 설정 객체 초기화
config = {"configurable": {"thread_id": "1"}}

# 첫 번째 사용자 메시지 생성 및 출력
input_message = HumanMessage(content="이틀 후부터 그 다음 5일 동안 '알바'라는 이름의 일정을 그룹 '일상'으로 추가해. 시간대는 12:00부터 18:00까지.")
input_message.pretty_print()

inputs = {"input": input_message.content}

# 스트림 모드에서 첫 번째 메시지 처리 및 업데이트 출력
for event in app.stream(inputs, config, stream_mode="updates"):
    print_update(event)


================================ Human Message =================================

이틀 후부터 그 다음 5일 동안 '알바'라는 이름의 일정을 그룹 '일상'으로 추가해. 시간대는 12:00부터 18:00까지.
--------------------
Plan step: GetNowDatetime
Plan step: CalculateDatetimeWithNumberOfDays(date=현재_날짜, number_of_days=2)
Plan step: CreateTasks(tasks='[["알바", "계산된_날짜 12:00 계산된_날짜 18:00", "일상"], ["알바", "계산된_날짜+1 12:00 계산된_날짜+1 18:00", "일상"], ["알바", "계산된_날짜+2 12:00 계산된_날짜+2 18:00", "일상"], ["알바", "계산된_날짜+3 12:00 계산된_날짜+3 18:00", "일상"], ["알바", "계산된_날짜+4 12:00 계산된_날짜+4 18:00", "일상"]]')
--------------------
Past step: ['ShowTasks', "현재 등록된 작업 목록은 다음과 같습니다:\n\n1. **[Squarekur 회의]** - 2025-06-22 22:00 (이벤트)\n2. **[Squarekur 회의]** - 2025-06-15 22:00 (일과)\n3. **[정보처리기사 최종합격자 발표]** - 2025-12-24 09:00 (이벤트)\n4. **[정보처리기사 실기시험]** - 2025-11-01 00:00 ~ 2025-11-21 00:00 (이벤트)\n5. **[정보처리기사 실기접수]** - 2025-09-22 10:00 ~ 2025-09-25 18:00 (이벤트)\n6. **[정보처리기사 필기합격 발표]** - 2025-09-10 09:00 (이벤트)\n7. **[정보처리기사 필기시험]** - 2025-08-09 00:00 ~ 2025-09-01 00:00 (

In [5]:
# 메시지 핸들링을 위한 HumanMessage 클래스 임포트
from langchain_core.messages import HumanMessage

# 스레드 ID가 포함된 설정 객체 초기화
config = {"configurable": {"thread_id": "1"}}

# 첫 번째 사용자 메시지 생성 및 출력
input_message = HumanMessage(content="'알바' 일정을 내일부터 3일까지 12:00부터 18:00까지 '일상'으로 추가해")
input_message.pretty_print()

inputs = {"input": input_message.content}

# 스트림 모드에서 첫 번째 메시지 처리 및 업데이트 출력
for event in app.stream(inputs, config, stream_mode="updates"):
    print_update(event)


================================ Human Message =================================

'알바' 일정을 내일부터 3일까지 12:00부터 18:00까지 '일상'으로 추가해
--------------------
Plan step: Get the current date and time using the GetNowDatetime tool.
Plan step: Calculate the date for tomorrow by adding 1 day to the current date using the CalculateDatetimeWithNumberOfDays tool.
Plan step: Create a task for '알바' from 12:00 to 18:00 for tomorrow under the '일상' group using the CreateTask tool.
Plan step: Calculate the date for the day after tomorrow by adding 2 days to the current date using the CalculateDatetimeWithNumberOfDays tool.
Plan step: Create a task for '알바' from 12:00 to 18:00 for the day after tomorrow under the '일상' group using the CreateTask tool.
Plan step: Calculate the date for three days from today by adding 3 days to the current date using the CalculateDatetimeWithNumberOfDays tool.
Plan step: Create a task for '알바' from 12:00 to 18:00 for three days from today under the '일상' group using the CreateTa

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [ ]:
# 유저에게 쿼리를 받고, 어떻게 처리할지 미리 계획을 세워두고 그 계획에 맞게 정보를 수집하고 처라히는 방식으로 그래프 구성하기